In [101]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [102]:
column_names = ["ID","Product_Name", "Total_votes", "Website_Link","Total_Reviews","Review_Score","Hunters","Founders","Industry_Type"]
df = pd.DataFrame(columns = column_names)

In [103]:
#df.to_csv('Prod_data.csv', mode='a', header=True,index=False)

In [104]:
service = Service("C:/Users/DELL/Downloads/chromedriver_win32/chromedriver.exe")
driver = webdriver.Chrome(service=service)

In [105]:
wait=WebDriverWait(driver, 5)

In [106]:
driver.maximize_window()
driver.implicitly_wait(20)

driver.get("https://www.producthunt.com/")

In [ ]:
lst_products_links=[]
for section in range(0,80):
    container = 'homepage-section-'+str(section)
    view_product = driver.find_element(By.XPATH,"//div[@data-test='"+container+"']//button[text()='View all products']")
    driver.execute_script("arguments[0].scrollIntoView();", view_product)
    driver.execute_script("arguments[0].click();", view_product)
    show_less = driver.find_element(By.XPATH,"//div[@data-test='"+container+"']//button[text()='Show less']")
    driver.execute_script("arguments[0].scrollIntoView();", show_less)
    lst_products_page = driver.find_elements(By.XPATH,"//div[@data-test='"+container+"']//a[contains(@data-test,'tagline')]")
    print(len(lst_products_page))
    for item in lst_products_page:
        lst_products_links.append(item.get_attribute('href'))
print(len(lst_products_links))

In [107]:
i=0
ID=2437
for item in lst_products_links[2437:]:
    driver.get(item)
    wait.until(EC.presence_of_element_located((By.XPATH, "//h1[contains(@class,'headerPostName')]//a[contains(@href,'/r/')]")))
    Prod_name=driver.find_element(By.XPATH,"//h1[contains(@class,'headerPostName')]//a[contains(@href,'/r/')]")
    df.loc[i,"Product_Name"]=Prod_name.text
    print("Product Name ",Prod_name.text)
    
    df.loc[i,"ID"]=ID


    wait.until(EC.presence_of_element_located((By.XPATH, "//span[@class='styles_bigButtonCount__1DS7y']")))
    Total_Votes=driver.find_element(By.XPATH,"//span[@class='styles_bigButtonCount__1DS7y']")
    df.loc[i,"Total_votes"]=Total_Votes.text



    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//*[text()='get it']/parent::a")))
        Website=driver.find_element(By.XPATH,"//*[text()='get it']/parent::a")
        df.loc[i,"Website_Link"]=Website.get_attribute('href')
    except :
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//a[@class='styles_link__2efWB']")))
            Website=driver.find_element(By.XPATH,"//a[@class='styles_link__2efWB']")           
            df.loc[i,"Website_Link"]=Website.get_attribute('href')    
        except :
            try:
                wait.until(EC.presence_of_element_located((By.XPATH, "//*[text()='get it']"))).click()
                wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='styles_allLinks__2sH2S']//a[1]")))
                Website=driver.find_element(By.XPATH,"//div[@class='styles_allLinks__2sH2S']//a[1]")
                df.loc[i,"Website_Link"]=Website.get_attribute('href')
            except:
                #print("Link not present")
                df.loc[i,"Website_Link"]="Link not present"




    try:
        Count_Reviews = driver.find_element(By.XPATH,"//span[contains(@class,'styles_reviewCountActive')]")
        df.loc[i,"Total_Reviews"]=Count_Reviews.text
    except :
        #print("Total Reviews : No reviews until now")
        df.loc[i,"Total_Reviews"]="No reviews until now"
    


    try:
        Review_Score = driver.find_element(By.XPATH,"//span[contains(@class,'styles_rating')]")
        df.loc[i,"Review_Score"]=Review_Score.text
    except :
        df.loc[i,"Review_Score"]="No Review Score until now"


    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='styles_makersContainer__1N77x'][1]//div[@class='styles_content__2OcVn']//div")))
        lst_Hunters = driver.find_elements(By.XPATH,"//div[@class='styles_makersContainer__1N77x'][1]//div[@class='styles_content__2OcVn']//div")
        Hunters=[]
        for item in lst_Hunters:
            Hunters.append(item.text)
        df.at[i,'Hunters']=Hunters
    except:
        df.at[i,'Hunters']="Hunter Not found"



    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='styles_makersContainer__1N77x'][2]//div[@class='styles_content__2OcVn']//div")))
        lst_Founders = driver.find_elements(By.XPATH,"//div[@class='styles_makersContainer__1N77x'][2]//div[@class='styles_content__2OcVn']//div")
        Founders=[]
        for item in lst_Founders:
            Founders.append(item.text)
        df.loc[i,"Founders"]=Founders            
    except:
        df.loc[i,"Founders"]="No Founders exist yet"  


    try:
        lst_Industry_type=[]
        elemnt1=driver.find_element(By.XPATH,"//div[@class='styles_topicPriceWrap__2fqZ7']//a[contains(@href,'/topics')]")
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='styles_topicPriceWrap__2fqZ7']//div/a[@title='More options']")))
        driver.find_element(By.XPATH,"//div[@class='styles_topicPriceWrap__2fqZ7']//div/a[@title='More options']").click()
        wait.until(EC.presence_of_element_located((By.XPATH, "//li[@class='styles_option__b7Uwf']//a[contains(@href,'/topics')]")))
        Others = driver.find_elements(By.XPATH,"//li[@class='styles_option__b7Uwf']//a[contains(@href,'/topics')]")
        lst_Industry_type.extend(Others)
        lst_Industry_type.append(elemnt1)
        Industry_type=[]
        for item in lst_Industry_type:
            Industry_type.append(item.text)
        df.loc[i,"Industry_Type"]=Industry_type
    except:
        df.loc[i,"Industry_Type"]='No Industry type'
 
    
    ID=ID+1
    df.to_csv('Prod_data_1.csv',mode='a',index=False,header=False)

Product Name  Uniify
Product Name  Ecohilo
Product Name  Metups
Product Name  TalkNow
Product Name  Pensso
Product Name  Bootstrappers
Product Name  Multiverse
Product Name  Pixelied
Product Name  Stately Viz
Product Name  Warmer.ai
Product Name  GuideDoc 2.0
Product Name  Smart Ads by Unbounce
Product Name  Laconic
Product Name  Hightouch Audiences
Product Name  Commont
Product Name  My Crypto Profile
Product Name  Icon Horse
Product Name  Walden Directory
Product Name  Sticker Doodle
Product Name  Noteli
Product Name  Bits 2.0 by Creative Tim
Product Name  Oka Todo
Product Name  LaserShow Video
Product Name  Dataherald by HiGeorge
Product Name  WellPaper
Product Name  GUN 2.0
Product Name  WorkPatterns for Zoom Apps
Product Name  Tweet Cards
Product Name  UI Copy plugin for XD
Product Name  Fart Sender
Product Name  HiHello Vaccine QR Code
Product Name  temper
Product Name  Dark Mode For Medium
Product Name  Ponder
Product Name  AppSecEngineer
Product Name  Cevoid
Product Name  Team 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00593AB3+2505395]
	Ordinal0 [0x0052AE41+2076225]
	Ordinal0 [0x00432498+1057944]
	Ordinal0 [0x0041BE96+966294]
	Ordinal0 [0x00477F19+1343257]
	Ordinal0 [0x004848C2+1394882]
	Ordinal0 [0x004758BB+1333435]
	Ordinal0 [0x004523E4+1188836]
	Ordinal0 [0x0045323F+1192511]
	GetHandleVerifier [0x0071CB36+1554566]
	GetHandleVerifier [0x007C4A0C+2242396]
	GetHandleVerifier [0x00620E0B+523099]
	GetHandleVerifier [0x0061FEB0+519168]
	Ordinal0 [0x005302FD+2097917]
	Ordinal0 [0x00534388+2114440]
	Ordinal0 [0x005344C2+2114754]
	Ordinal0 [0x0053E041+2154561]
	BaseThreadInitThunk [0x7648FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77037A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77037A6E+238]


In [ ]:
print(i)

In [ ]:
print(df)

In [61]:
df1=df

In [62]:
df.to_excel (r'Data5.xlsx', index = False, header=True)